---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

import matplotlib.pyplot as plt
#fig, axes = plt.subplots(1,5, figsize=(10,15))
for i in range(len(P1_Graphs)):
    degree = P1_Graphs[i].degree()
    degree_values=sorted(set(degree.values()))
    hist = [list(degree.values()).count(j)/float(nx.number_of_nodes(items)) for j in degree_values]
    #plt.subplot(5,1,(i+1)%5)
    plt.plot(degree_values, hist, 'o')
    plt.xlabel('Degree of graph {}'.format(i))
    plt.ylabel('Fraction of nodes')
    #plt.xscale('log')
    #plt.yscale('log')
    plt.show()

list1=[]
x= range(len(P1_Graphs))
for items in P1_Graphs:
    list1.append(nx.average_clustering(items))
plt.bar(x, list1)
plt.show()

In [3]:
def graph_identification():    
    return ['PA', 'SW_L', 'SW_L','PA','SW_H']
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
df=pd.DataFrame(index=G.nodes())
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['Salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['clustering'] = pd.Series(nx.clustering(G))
df['degree'] = pd.Series(G.degree())
df['degree_centrality'] = [val for num, val in nx.degree_centrality(G).items()]
df['closeness_centrality'] = [val for num, val in nx.closeness_centrality(G).items()]
df['between_centrality'] = [val for num, val in nx.betweenness_centrality(G).items()]

In [6]:
def salary_predictions():
    test =df[np.isnan(df['Salary'])]
    test.drop('Salary', axis=1, inplace=True)
    train = df[np.isfinite(df['Salary'])]
    cols = train.columns
    new_cols = ['Department', 'clustering', 'degree', 'degree_centrality',
           'closeness_centrality', 'between_centrality', 'Salary']
    train = train[new_cols]
   
    X=train[['Department', 'clustering', 'degree', 'degree_centrality',
       'closeness_centrality', 'between_centrality']]
    y=train[['Salary']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

    clf = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2).fit(X_train, y_train)

    predict = clf.predict(X_test)
    roc_auc_score(predict, y_test)
    result= clf.predict_proba(test)
    final = pd.DataFrame(data=result, index=test.index)

    return final[1]
salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1       0.140424
2       0.686282
5       0.686282
8       0.198548
14      0.144430
18      0.144430
27      0.144430
30      0.216365
31      0.198548
34      0.100777
37      0.098848
40      0.140424
45      0.140424
54      0.198548
55      0.216365
60      0.150857
62      0.686282
65      0.686282
77      0.145721
79      0.144430
97      0.098848
101     0.067454
103     0.216365
108     0.144430
113     0.144430
122     0.067454
141     0.331077
142     0.686282
144     0.098848
145     0.216365
          ...   
913     0.098848
914     0.098848
915     0.067454
918     0.100777
923     0.100777
926     0.100777
931     0.098848
934     0.067454
939     0.067454
944     0.067454
945     0.100777
947     0.100777
950     0.144430
951     0.098848
953     0.067454
959     0.067454
962     0.067454
963     0.198548
968     0.100777
969     0.100777
974     0.100777
984     0.067454
987     0.100777
989     0.100777
991     0.100777
992     0.067454
994     0.067454
996     0.0674

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [41]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head()

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [43]:
def new_connections_predictions():
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.linear_model import LogisticRegression
    future_connections['preferential attachment'] = [i[1] for i in nx.preferential_attachment(G, future_connections.index)]
    list1 =[]
    for i in range(len(future_connections)):
        list1.append(len(list(nx.common_neighbors(G, future_connections.index[i][0],future_connections.index[i][1]))))
    future_connections['common neighbors']= list1
    future_connections['jaccard'] = [p for u, v, p in list(nx.jaccard_coefficient(G, list(i for i in future_connections.index)))]
    test = future_connections[np.isnan(future_connections['Future Connection'])]
    train = future_connections[np.isfinite(future_connections['Future Connection'])]
    test.drop('Future Connection', axis=1, inplace=True)
    cols = ['preferential attachment',  'common neighbors','jaccard']
    X = train[cols]
    y = train['Future Connection']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    clf = LogisticRegression().fit(X_train, y_train)
    predict = clf.predict(X_test)
    roc_auc_score(predict, y_test)
    result = clf.predict_proba(test)
    final = pd.DataFrame(data=result, index=test.index)
    return final[1]
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.029613
(542, 751)    0.014387
(20, 426)     0.675821
(50, 989)     0.017132
(942, 986)    0.017094
(324, 857)    0.015551
(13, 710)     0.241932
(19, 271)     0.187830
(319, 878)    0.015793
(659, 707)    0.013929
(49, 843)     0.015392
(208, 893)    0.015968
(377, 469)    0.011691
(405, 999)    0.028701
(129, 740)    0.023762
(292, 618)    0.021856
(239, 689)    0.013746
(359, 373)    0.010892
(53, 523)     0.033930
(276, 984)    0.017069
(202, 997)    0.017233
(604, 619)    0.037841
(270, 911)    0.016180
(261, 481)    0.087688
(200, 450)    0.782770
(213, 634)    0.013201
(644, 735)    0.040400
(346, 553)    0.012437
(521, 738)    0.014250
(422, 953)    0.027849
                ...   
(672, 848)    0.015449
(28, 127)     0.930525
(202, 661)    0.013466
(54, 195)     0.999386
(295, 864)    0.015631
(814, 936)    0.016479
(839, 874)    0.015747
(139, 843)    0.015392
(461, 544)    0.012355
(68, 487)     0.011847
(622, 932)    0.016431
(504, 936)    0.027488
(479, 528) 